In [16]:
import os, re
import cdsapi
import geopandas as gpd
from shapely.geometry import Polygon, mapping
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from timezonefinder import TimezoneFinder
import pytz
from zoneinfo import ZoneInfo

In [2]:
# To get root path under project folder structure
cwd = os.path.dirname(os.path.abspath("__file__"))
root_folder = cwd.split('\\')
root_folder = root_folder[1:-3]
if root_folder[-1] != 'fireRunSeverity':
    print("!!-- Didn't get correct root folder! --!!")
    print(root_folder)
    print("!!-- Modify rooting index at line: 18 and come back --!!")
root_folder = os.path.join(os.sep,*root_folder)
run_DataDir = r"data/fireruns"
ER5_Dir     = r"data/ER5"

# Local Side data
# List in put data for GEE fetch
AreaList = os.listdir(os.path.join(root_folder,run_DataDir))
AreaList[0]
os.path.exists(os.path.join(root_folder, ER5_Dir))

True

## Load Local side data

In [7]:
in_Name = AreaList[0]
print("\n\n ######################################################")
print("Process AOI:", in_Name)
dataFLD = os.path.join(root_folder, run_DataDir, in_Name)
inFLD = os.path.join(dataFLD, 'input')
shpIn  = []
for f in os.scandir(inFLD):
    if re.search(r".shp$", f.name):
        print("Shp Name:", f.name)
        shpIn.append(f)

if len(shpIn) != 1:
    print("No. of shp file not equal to 1!")
    exit()
else:
    shpIn = shpIn[0]

# Read Shp file
shpGPD = gpd.read_file(os.path.join(inFLD, shpIn))
print("Shp  CRS:", shpGPD.crs)
shp_reproj = shpGPD.to_crs("EPSG:4326")
# Get the bounds
minx, miny, maxx, maxy = shp_reproj.total_bounds
# Create a polygon of bounds
bounds_polygon = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])
# Convert the polygon to GeoJSON
json_polygon = mapping(bounds_polygon)
print(json_polygon)



 ######################################################
Process AOI: GIF14_Au
Shp Name: GIF14_32755_multipart.shp
Shp  CRS: EPSG:32755
{'type': 'Polygon', 'coordinates': (((148.797548490504, -37.78877807241547), (148.797548490504, -37.04608874283253), (150.0295287036503, -37.04608874283253), (150.0295287036503, -37.78877807241547), (148.797548490504, -37.78877807241547)),)}


## Modify time zone

In [9]:
def get_timezone(lat, lon):
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lng=lon, lat=lat)
    return tz_name

latitude = (miny + maxy)/2
longitude = (minx + maxx)/2
timezone = get_timezone(latitude, longitude)
print(f"Accurate Time Zone: {timezone}")

Accurate Time Zone: Australia/Melbourne


In [27]:
for in_Name in AreaList:
    print("Process AOI:", in_Name, end="\t\t\t")
    dataFLD = os.path.join(root_folder, run_DataDir, in_Name)
    inFLD = os.path.join(dataFLD, 'input')
    shpIn  = []
    for f in os.scandir(inFLD):
        if re.search(r".shp$", f.name):
            # print("Shp Name:", f.name)
            shpIn.append(f)

    if len(shpIn) != 1:
        print("No. of shp file not equal to 1!")
        exit()
    else:
        shpIn = shpIn[0]

    shpGPD = gpd.read_file(os.path.join(inFLD, shpIn))

    # Get date of data
    # Convert time zone to UTC
    fire_dtString = list(shpGPD['FeHo'].dropna())
    fire_dtsList  = [datetime.strptime(dt, "%Y/%m/%d_%H%M").replace(tzinfo=ZoneInfo(timezone)) for dt in fire_dtString if not re.search("\\D", dt[:4])] # Filter out string with non-digit characters
    tz_utc = pytz.timezone("UTC")
    fire_dtsList_UTC = [dt.astimezone(tz_utc) for dt in fire_dtsList]

    # print(sorted(fire_dtsList_UTC))
    dateSt = min(fire_dtsList_UTC).date()
    print(dateSt.year, dateSt.month)

Process AOI: GIF14_Au			2019 12
Process AOI: isocronas_las_tablas			2024 2
Process AOI: LaJonquera			2012 7
Process AOI: LC1			2020 8
Process AOI: santa_ana_progresiones			2023 2
Process AOI: StLlorenc			2003 8
Process AOI: GIF11_Au			2019 12
Process AOI: GIF12_Au			2019 12
Process AOI: GIF15_Au			2019 12
Process AOI: LasMaquinas			2017 1
Process AOI: LC2			2020 8
Process AOI: PT_Alcobaca_15102017			2017 10
Process AOI: PT_Gois_17062017			2017 6
Process AOI: PT_Lousa_15102017			2017 10
Process AOI: PT_Monchique_03082018			2018 8
Process AOI: PT_PedrogaoGrande_17062017			2017 6
Process AOI: PT_Serta_23072017			2017 7
Process AOI: PT_FigueiradaFoz_15102017			2017 10
Process AOI: PT_OliveiraFrades_15102017			2017 10
Process AOI: PT_ParedesdeCoura_07082016			2016 8
Process AOI: PT_ProencaaNova_13092020			2020 9
Process AOI: PT_ViladeRei_20072019			2019 7


In [19]:
# Get date of data
# Convert time zone to UTC
fire_dtString = list(shpGPD['FeHo'].dropna())
fire_dtsList  = [datetime.strptime(dt, "%Y/%m/%d_%H%M").replace(tzinfo=ZoneInfo(timezone)) for dt in fire_dtString if not re.search("\\D", dt[:4])] # Filter out string with non-digit characters
tz_utc = pytz.timezone("UTC")
fire_dtsList_UTC = [dt.astimezone(tz_utc) for dt in fire_dtsList]

# print(sorted(fire_dtsList_UTC))
dateSt = min(fire_dtsList_UTC).date()
dateEd = max(fire_dtsList_UTC).date()
print("Fire start date and end date:")
print(dateSt, dateEd)

# Compute the date period for retrieving Satellite img
prefire_Month  = [dateSt - relativedelta(months = 30*12), dateSt]
prefire_Month
# prefire_date = [d.strftime("%Y-%m-%d") for d in prefire_date]

# print("Pre- and Post- fire periods selected for satellite images:")
# print(prefire_date, postfire_date)

Fire start date and end date:
2019-12-29 2020-02-07


[datetime.date(1989, 12, 29), datetime.date(2019, 12, 29)]

In [33]:
date(dateSt.year, dateSt.month, 1), date(dateSt.year, dateSt.month, 30)


(datetime.date(2019, 7, 1), datetime.date(2019, 7, 30))

In [ ]:
var_weather = ["total_precipitation", "potential_evaporation"]
dataset = "reanalysis-era5-single-levels-monthly-means"

for var in var_weather:
    request = {
        "product_type": ["monthly_averaged_reanalysis"],
        "variable": [var],
        "year": ["1990", "2024"],
        "month": [
            "01", "02", "03",
            "04", "05", "06",
            "07", "08", "09",
            "10", "11", "12"
        ],
        "time": ["00:00"],
        "data_format": "grib",
        "download_format": "unarchived",
        "area": [90, -180, -39, 180]
    }

    client = cdsapi.Client()
    client.retrieve(dataset, request).download()


In [ ]:
var_weather = ["2m_temperature","2m_dewpoint_temperature","10m_u_component_of_wind","10m_v_component_of_wind"]

for var in var_weather:
    os.makedirs(os.path.join(processed_path, "Weather", "CDS", var),
                exist_ok=True)

if fetch:
    c = cdsapi.Client(key=cdsAPI_KEY,
                    url='https://cds.climate.copernicus.eu/api', quiet=False, sleep_max=5, timeout=7200)

    firstRun = True
    for var in var_weather:
        print(f"\n...Downloading {var}...")
        if os.path.exists(os.path.join(processed_path,"Weather", f"era5_{var}.nc")):
            print("Exists: ", os.path.join(processed_path,"Weather", f"era5_{var}.nc"))
            continue

        downloaded_files = []
        print(f"Requesting......")
        for (year, month), dayRange in dates_by_month.items():
            print(f"{year}-{month}", end=" ")
            output_file = os.path.join(processed_path, "Weather", "CDS", var,
                                        f"era5_land_{year}_{month}_{dayRange[0]}-{dayRange[1]-1}_12UTC.nc")
            if os.path.exists(output_file):
                continue

            c.retrieve(
                'reanalysis-era5-land',
                {
                    "variable": [var],
                    'year': year,
                    'month': month,
                    'day': [f"{i:02}" for i in range(*dayRange)],
                    'time': time_of_day,
                    "area": area_extract_cds,
                    "data_format": "netcdf",
                    "download_format": "unarchived",
                },
                output_file
            )

            if firstRun:
                c = cdsapi.Client(key=cdsAPI_KEY, 
                        url='https://cds.climate.copernicus.eu/api', quiet=True, sleep_max=5, timeout=7200, progress=False)
                firstRun = False

            downloaded_files.append(output_file)

        print("\nMerging NetCDF files...")
        merged_file = os.path.join(processed_path, "Weather", f"era5_{var}.nc")
        if os.path.exists(merged_file):
            print(f"File already exists: {merged_file}")
            continue

        datasets = [xr.open_dataset(f, engine='netcdf4') for f in downloaded_files]
        merged = xr.concat(datasets, dim='valid_time')
        merged.to_netcdf(merged_file)
        print(f"Saved merged NetCDF as {merged_file}\n")

    print("Extraction from CDS API finished")
    return True
else:
    print("Weather data not fetched")
    return False

In [ ]:
dataset = "satellite-land-cover"
request = {
  "variable": "all",
  "year": [
    "2014", "2017", "2019",
    "2022"
  ],
  "version": [
    "v2_0_7cds",
    "v2_1_1"
  ],
  "area": [90, -180, -90, 180]
}